## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop('IS_SUCCESSFUL', axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 352       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
#advise from jeremy tallant
# create directory if it does not exist
if not os.path.exists('models\model_1'):
    os.makedirs('models\model_1')
# Create a callback that saves the model's weights every five epochs
checkpoint_path = "models\model_1\weights.{epoch:02d}.hdf5"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq='epoch', period=5)

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 1s 955us/step - loss: 0.6541 - accuracy: 0.6515
Epoch 2/100
804/804 [==============================] - 1s 893us/step - loss: 0.5730 - accuracy: 0.7207
Epoch 3/100
804/804 [==============================] - 1s 891us/step - loss: 0.5634 - accuracy: 0.7243
Epoch 4/100
804/804 [==============================] - 1s 890us/step - loss: 0.5596 - accuracy: 0.7277
Epoch 5/100
782/804 [============================>.] - ETA: 0s - loss: 0.5571 - accuracy: 0.7281
Epoch 5: saving model to models\model_1\weights.05.hdf5
804/804 [==============================] - 1s 923us/step - loss: 0.5571 - accuracy: 0.7282
Epoch 6/100
804/804 [==============================] - 1s 998us/step - loss: 0.5546 - accuracy: 0.7287
Epoch 7/100
804/804 [==============================] - 1s 921us/step - loss: 0.5534 - accuracy: 0.7282
Epoch 8/100
804/804 [==============================] - 1s 875us/step - loss: 0.5528 - accuracy: 0.7288
Epoch 9/100
804/804 [==============

804/804 [==============================] - 1s 882us/step - loss: 0.5453 - accuracy: 0.7315
Epoch 64/100
804/804 [==============================] - 1s 878us/step - loss: 0.5453 - accuracy: 0.7318
Epoch 65/100
773/804 [===========================>..] - ETA: 0s - loss: 0.5446 - accuracy: 0.7330
Epoch 65: saving model to models\model_1\weights.65.hdf5
804/804 [==============================] - 1s 865us/step - loss: 0.5451 - accuracy: 0.7324
Epoch 66/100
804/804 [==============================] - 1s 924us/step - loss: 0.5450 - accuracy: 0.7325
Epoch 67/100
804/804 [==============================] - 1s 829us/step - loss: 0.5451 - accuracy: 0.7324
Epoch 68/100
804/804 [==============================] - 1s 838us/step - loss: 0.5451 - accuracy: 0.7325
Epoch 69/100
804/804 [==============================] - 1s 828us/step - loss: 0.5451 - accuracy: 0.7328
Epoch 70/100
786/804 [============================>.] - ETA: 0s - loss: 0.5455 - accuracy: 0.7329
Epoch 70: saving model to models\model_1\weig

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5522 - accuracy: 0.7338 - 277ms/epoch - 1ms/step
Loss: 0.5521971583366394, Accuracy: 0.7337609529495239


In [17]:
# Export our model to HDF5 file
nn.save('models/model_1/AlphabetSoupCharity.h5')